Привет!
Высылаю последнее задание по курсу.
Спойлер: Решил так, как понял. Пришлось перечитать несколько раз, чтобы понять, что именно ожидается как результат. Очень большая просьба в будущем давать задания понятнее, так как на курсе разных уровень подготовки у студентов. 🫀

In [1]:
!pip install -q -U kaggle_environments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [3]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Writing rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [4]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [5]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [25]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

Описание агентов:

`rock_agent.py`– агент, всегда выбирающий "камень".

`paper_agent.py` – агент, всегда выбирающий "бумагу".

`scissors_agent.py` – агент, всегда выбирающий "ножницы".

`random_agent.py` – агент, который выбирает случайный ход на каждом шаге.

`copy_opponent.py` – агент, который копирует действие противника.

`cycle_agent.py` – агент, который по очереди выбирает "камень", "бумагу" и "ножницы" циклически.

`anti_copy_agent.py` – агент, который всегда выбирает контр-ход на предыдущий ход противника (выигрывающий ход).

`biased_rock_agent.py` – агент, который с большей вероятностью выбирает "камень", но иногда случайно выбирает другие действия.

`mirror_agent.py` – агент, который на текущем шаге делает ход, который противник сделал 2 шага назад.

`opposite_last_agent.py` – агент, который всегда выбирает противоположный ход к предыдущему ходу противника.

`frequency_counter_agent.py` – агент, который анализирует, какое действие противник использует чаще всего, и выбирает контр-стратегию.

`random_pattern_agent.py` – агент, который случайно выбирает одно из заранее заданных последовательностей действий (например, [0, 1, 2] или [2, 0, 1]).

`win_streak_agent.py` – агент, который повторяет свой последний выигрышный ход, если он побеждает, или делает случайный ход, если проигрывает.

In [7]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1  # всегда выбирает "бумагу"

Writing paper_agent.py


In [8]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2  # всегда выбирает "ножницы"

Writing scissors_agent.py


In [9]:
%%writefile random_agent.py

import random
def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)  # случайный выбор


Writing random_agent.py


In [10]:
%%writefile cycle_agent.py

def cycle_agent(observation, configuration):
    return observation.step % 3  # циклический выбор "камень", "бумага", "ножницы"


Writing cycle_agent.py


In [11]:
%%writefile anti_copy_agent.py

def anti_copy_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % 3  # выбирает контр-ход
    else:
        return random.randrange(0, configuration.signs)


Writing anti_copy_agent.py


In [12]:
%%writefile biased_rock_agent.py

import random
def biased_rock_agent(observation, configuration):
    # 70% вероятность выбора "камень", 30% случайного выбора
    return 0 if random.random() < 0.7 else random.randrange(0, configuration.signs)


Writing biased_rock_agent.py


In [13]:
%%writefile mirror_agent.py

def mirror_agent(observation, configuration):
    if observation.step >= 2:
        return observation['agentReward'][-2]  # повторяет ход 2 шага назад
    else:
        return random.randrange(0, configuration.signs)


Writing mirror_agent.py


In [14]:
%%writefile opposite_last_agent.py

def opposite_last_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % 3  # противоположный ход к последнему
    else:
        return random.randrange(0, configuration.signs)


Writing opposite_last_agent.py


In [15]:
%%writefile frequency_counter_agent.py

def frequency_counter_agent(observation, configuration):
    if observation.step == 0:
        return random.randrange(0, configuration.signs)

    counts = [0, 0, 0]
    for i in range(observation.step):
        counts[observation.lastOpponentAction] += 1
    return (counts.index(max(counts)) + 1) % 3  # выбирает ход против самого частого действия


Writing frequency_counter_agent.py


In [16]:
%%writefile win_streak_agent.py

import random

def win_streak_agent(observation, configuration):
    if observation.step == 0:
        return random.randrange(0, configuration.signs)

    if observation.reward > 0:  # если выиграл, повторяет свой последний ход
        return observation.agentReward
    else:
        return random.randrange(0, configuration.signs)  # случайный ход, если проиграл

Writing win_streak_agent.py


In [17]:
%%writefile random_pattern_agent.py

import random

def random_pattern_agent(observation, configuration):
    patterns = [
        [0, 1, 2],  # камень, бумага, ножницы
        [1, 2, 0],  # бумага, ножницы, камень
        [2, 0, 1]   # ножницы, камень, бумага
    ]
    pattern = random.choice(patterns)
    return pattern[observation.step % len(pattern)]  # выбирает ход из случайного паттерна


Writing random_pattern_agent.py


In [18]:
%%writefile counter_random_agent.py

import random

def counter_random_agent(observation, configuration):
    # против случайного агента вероятности равны для каждого хода
    # мы выбираем случайный контр-ход
    return random.choice([0, 1, 2])


Writing counter_random_agent.py


In [34]:
def last_two_opponent_moves_agent(observation, configuration):
    # Если есть информация о предыдущем ходе
    if observation.step == 0:
        return 0  # Выбор первого хода (например, камень)

    # Инициализируем историю ходов
    if not hasattr(last_two_opponent_moves_agent, "last_moves"):
        last_two_opponent_moves_agent.last_moves = [None, None]

    # Обновляем историю ходов
    last_two_opponent_moves_agent.last_moves[0] = last_two_opponent_moves_agent.last_moves[1]
    last_two_opponent_moves_agent.last_moves[1] = observation.lastOpponentAction

    # Если соперник уже сделал два хода
    if last_two_opponent_moves_agent.last_moves[0] is not None:
        # Выбираем ход на основе последних двух ходов соперника
        return (last_two_opponent_moves_agent.last_moves[1] + 1) % 3  # counter-move
    else:
        return 0  # Если информации недостаточно, выбираем камень


In [35]:
from kaggle_environments import evaluate

# Список всех агентов с их именами файлов
agents = [
    "rock_agent.py",            # Агент, который всегда выбирает камень
    "paper_agent.py",           # Агент, который всегда выбирает бумагу
    "scissors_agent.py",        # Агент, который всегда выбирает ножницы
    "random_agent.py",          # Агент, который выбирает случайный ход
    "cycle_agent.py",           # Агент, который делает ход по кругу
    "anti_copy_agent.py",       # Агент, который противодействует предыдущему ходу
    "biased_rock_agent.py",     # Агент, который делает больший акцент на камень
    "mirror_agent.py",          # Агент, который копирует ход соперника
    "opposite_last_agent.py",   # Агент, который выбирает противоположный ход соперника
    "frequency_counter_agent.py", # Агент, который выбирает ход на основе статистики предыдущих ходов
    "win_streak_agent.py",      # Агент, который пытается выиграть по несколько раз подряд
    "counter_random_agent.py",  # Агент, который реагирует на случайные ходы соперника
    # "last_two_opponent_moves_agent.py"  # Исключите проблемный агент
]

# Результаты турнира
scoreboard = {agent: 0 for agent in agents}

# Провести турниры между всеми парами агентов
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        agent1 = agents[i]
        agent2 = agents[j]

        try:
            # Получаем результаты для 100 игр
            results = evaluate(
                "rps",  # среда для игры (камень-ножницы-бумага)
                [agent1, agent2],  # два агента
                configuration={"episodeSteps": 100}  # количество игр
            )

            # Суммируем результаты для каждого агента, игнорируя None
            total_score_agent1 = sum(result[0] for result in results if result[0] is not None)
            total_score_agent2 = sum(result[1] for result in results if result[1] is not None)

            scoreboard[agent1] += total_score_agent1
            scoreboard[agent2] += total_score_agent2

        except Exception as e:
            print(f"Ошибка при запуске агентов {agent1} и {agent2}: {e}")

# Вывод результатов турнира
print("Результаты турнира:")
for agent, score in scoreboard.items():
    print(f"{agent}: {score} очков")





Результаты турнира:
rock_agent.py: 5.0 очков
paper_agent.py: 78.0 очков
scissors_agent.py: -69.0 очков
random_agent.py: 5 очков
cycle_agent.py: 5 очков
anti_copy_agent.py: 0 очков
biased_rock_agent.py: 6.0 очков
mirror_agent.py: 0 очков
opposite_last_agent.py: 0 очков
frequency_counter_agent.py: 0 очков
win_streak_agent.py: 4 очков
counter_random_agent.py: 5 очков
